In [1]:
rm(list = ls())

library(imputeTS)
library(lubridate)
library(tictoc)
library(randomForest)
require(xts)
require(forecast)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following object is masked from ‘package:imputeTS’:

    na.locf

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: forecast


In [2]:
###################################
#
# Set Flags and Properties here
#
###################################

# To turn on validation
isValidation <- 0

# History folder name containing historical datasets
loadHistoricalFolderName <- "historicalWithLatest"

# Output Folder name for writing updated historical data with latest data
updateHistorical <- 1
writeHistoricalFolderName <- "historicalWithLatest"

# Flag to turn RF training ON/Off
trainRF <- 0
ntree <- 100
mtry <- 4

# Flag to turn SVR training ON/Off
trainSVR <- 0

# Flag to turn NN training ON/Off
trainANN <- 0
hidden_nodes <- 4 #hiddlen layer neurons
iterations <- 10
lr <- 0.05
stepmax <- 5000
error_func <- 'sse' #for regression

# Flag to predict RF or SVR.  "RF" - RF;  "SVR" - SVR; "ANN" - neural network
toPredict <- "SVR"
###################################

In [3]:
## Using custom callbacks in tic/toc
my.msg.tic <- function(tic, msg)
{
   if (is.null(msg) || is.na(msg) || length(msg) == 0)
   {
      outmsg <- paste(round(toc - tic, 3), " seconds elapsed", sep="")
   }
   else
   {
      outmsg <- paste("Starting ", msg, "...", sep="")
   }
}

my.msg.toc <- function(tic, toc, msg, info)
{
   if (is.null(msg) || is.na(msg) || length(msg) == 0)
   {
      outmsg <- paste(round(toc - tic, 3), " seconds elapsed", sep="")
   }
   else
   {
      outmsg <- paste(info, ": ", msg, ": ",
                   round(toc - tic, 3), " seconds elapsed", sep="")
   }
}
tic.clearlog()
tic("Total time", quiet = FALSE, func.tic = my.msg.tic)

Starting Total time...


In [4]:
tic("Load latest data", quiet = FALSE, func.tic = my.msg.tic)
# Load the data from the csv files
read.data = function(file = ''){
  ## Read the csv file
  air.quality <- read.csv(file, header = TRUE, 
                      stringsAsFactors = FALSE)
}

bj.aq.latest = read.data('bj_airquality_latest.csv')
ld.aq.latest = read.data('ld_airquality_latest.csv')
# Create a list of the beijing station ID names
bj_station_names_vec<-head(bj.aq.latest,35)$station_id
# Manually input the station names that we need for London
ld_station_names_vec <- c('CD1','BL0','GR4','MY7','HV1','GN3','GR9','LW2','GN0','KF1','CD9','ST5','TH4')

correct_col_names <- c('id', 'stationId', 'utc_time', 'PM2.5', 'PM10', 'NO2','CO', 'O3', 'SO2')

#rename bj and ld to latest column names
names(bj.aq.latest) <- correct_col_names
names(ld.aq.latest) <- correct_col_names

# Drop the id column of the latest bj aq file
bj.aq.latest <- bj.aq.latest[ , !(names(bj.aq.latest) %in% c("id"))]
# Drop the id column of the latest ld aq file
ld.aq.latest <- ld.aq.latest[, !(names(ld.aq.latest) %in% c("id", "CO", "O3", "SO2"))]

print(paste("number of latest aq rows of bj:",nrow(bj.aq.latest)))
print(paste("number of latest aq rows of ld:",nrow(ld.aq.latest)))

bj_station_names_vec
nrow(bj.aq.latest)/35

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Load latest data...
[1] "number of latest aq rows of bj: 1365"
[1] "number of latest aq rows of ld: 874"


[1] "dongsi_aq"         "tiantan_aq"        "guanyuan_aq"      
 [4] "wanshouxigong_aq"  "aotizhongxin_aq"   "nongzhanguan_aq"  
 [7] "wanliu_aq"         "beibuxinqu_aq"     "zhiwuyuan_aq"     
[10] "fengtaihuayuan_aq" "yungang_aq"        "gucheng_aq"       
[13] "fangshan_aq"       "daxing_aq"         "yizhuang_aq"      
[16] "tongzhou_aq"       "shunyi_aq"         "pingchang_aq"     
[19] "mentougou_aq"      "pinggu_aq"         "huairou_aq"       
[22] "miyun_aq"          "yanqin_aq"         "dingling_aq"      
[25] "badaling_aq"       "miyunshuiku_aq"    "donggaocun_aq"    
[28] "yongledian_aq"     "yufa_aq"           "liulihe_aq"       
[31] "qianmen_aq"        "yongdingmennei_aq" "xizhimenbei_aq"   
[34] "nansanhuan_aq"     "dongsihuan_aq"

[1] 39

INFO: Load latest data: 0.049 seconds elapsed


In [5]:
tic("Load Historical data", quiet = FALSE, func.tic = my.msg.tic)
bj.fullStationData <- list()
ld.fullStationData <- list()
for (s in 1:35) {
    bj.fullStationData[[s]] <- read.data(paste(loadHistoricalFolderName,"/",bj_station_names_vec[s],"_historical.csv",sep=""))
}

for (s in 1:13) {
    ld.fullStationData[[s]] <- read.data(paste(loadHistoricalFolderName,"/",ld_station_names_vec[s],"_historical.csv",sep=""))
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Load Historical data...
INFO: Load Historical data: 6.712 seconds elapsed


In [6]:
# Convert the bj date time from string to POSIX time in UTC in order to add missing rows
bj.aq.latest$utc_time <- as.POSIXct(bj.aq.latest$utc_time, format="%Y-%m-%d %H:%M:%S", tz="UTC")
# get starting and ending time
bj.startingTime <- head(bj.aq.latest$utc_time,1)
bj.startingTime
bj.endingTime <- tail(bj.aq.latest$utc_time,1)
bj.endingTime

#remove possible duplicate rows
bj.aq.latest <- unique(bj.aq.latest)
nrow(bj.aq.latest)

# Repeat for London
ld.aq.latest$utc_time <- as.POSIXct(ld.aq.latest$utc_time, format="%Y-%m-%d %H:%M:%S", tz="UTC")
# get starting and ending time
ld.startingTime <- head(ld.aq.latest$utc_time,1)
ld.startingTime
ld.endingTime <- tail(ld.aq.latest$utc_time,1)
ld.endingTime

#remove duplicate rows
ld.aq.latest <- unique(ld.aq.latest)
nrow(ld.aq.latest)

[1] "2018-05-30 08:00:00 UTC"

[1] "2018-05-31 22:00:00 UTC"

[1] 1365

[1] "2018-05-30 UTC"

[1] "2018-05-31 21:00:00 UTC"

[1] 872

In [7]:
# Find the difference between the latest bj first and last record
bj.diff <- as.integer(bj.endingTime - bj.startingTime + 1)  * 24 - hour(bj.startingTime) - (23 - hour(bj.endingTime))
bj.diff

# Repeat for London
ld.diff <- as.integer(ld.endingTime - ld.startingTime + 1)  * 24 - hour(ld.startingTime) - (23 - hour(ld.endingTime))
ld.diff

[1] 39

[1] 46

In [8]:
# Break the latest data into a list of dataframes for each station and sort by time
bj.stationData<-list()
for (i in 1:35) {
    bj.stationData[[i]]<-bj.aq.latest[bj.aq.latest$stationId==bj_station_names_vec[i],]
    bj.stationData[[i]] <- bj.stationData[[i]][order(bj.stationData[[i]]$utc_time),]
}

ld.stationData<-list()
for (i in 1:13) {
    ld.stationData[[i]]<-ld.aq.latest[ld.aq.latest$stationId==ld_station_names_vec[i],]
    ld.stationData[[i]] <- ld.stationData[[i]][order(ld.stationData[[i]]$utc_time),]
}

length(bj.stationData)
length(ld.stationData)

[1] 35

[1] 13

In [9]:
# Create a function that adds missing rows if time isn't continuous
addMissingRows <- function(stationData, timeDiff, colNames, startingTime) {
    
fullStationData <- list()
for (s in 1:length(stationData)) {
    #first create a dataframe with 'diff' number of rows
    fullStationData[[s]] <- as.data.frame(matrix(NA, nrow = timeDiff, ncol = length(colNames)))
    names(fullStationData[[s]]) <- colNames
    
    len_station_data <- nrow(stationData[[s]])
    new_index <- 1  #current index at fullStationData
    old_index <- 1  #current index at stationData
    fill_time <- startingTime
    while (new_index <= timeDiff && old_index <= len_station_data) {
       if (fill_time == stationData[[s]]$utc_time[old_index]) {
           # if same time, copy row from stationData to fullStationData
           fullStationData[[s]][new_index,] <- stationData[[s]][old_index,]
           # and increment both index
           old_index <- old_index + 1   
           
       } else {
           # if not the same time, then only increment new_index, and set stationId and utc_time
           fullStationData[[s]]$stationId[new_index] <- stationData[[s]]$stationId[1]
           fullStationData[[s]]$utc_time[new_index] <- fill_time
       }
        
        new_index <- new_index + 1
        fill_time <- fill_time + 3600 # increment time by 1 hour  
    }
    
    # fill fullStationData with the rest if there's any
    while (new_index <= timeDiff) {
        fullStationData[[s]]$stationId[new_index] <- stationData[[s]]$stationId[1]
        fullStationData[[s]]$utc_time[new_index] <- fill_time
        new_index <- new_index + 1
        fill_time <- fill_time + 3600
    }   
}
    
return(fullStationData)

}

In [10]:
tic("Fill missing rows for latest current data", quiet = FALSE, func.tic = my.msg.tic)
# fill missing rows for latest beijing and london aq data
bj.aq.latest.filled <- addMissingRows(bj.stationData, bj.diff, names(bj.aq.latest), bj.startingTime)
ld.aq.latest.filled <- addMissingRows(ld.stationData, ld.diff, names(ld.aq.latest), ld.startingTime)

# Verify that the number of rows is equal to the time diff now
nrow(bj.aq.latest.filled[[1]])
nrow(ld.aq.latest.filled[[1]])

rm(bj.aq.latest)
rm(ld.aq.latest)
rm(bj.stationData)
rm(ld.stationData)
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Fill missing rows for latest current data...


[1] 39

[1] 46

INFO: Fill missing rows for latest current data: 0.622 seconds elapsed


In [11]:
#fill in empty/missing data for each pollutant for each station using Kalman filters
tic("Fill in missing data with Kalman", quiet = FALSE, func.tic = my.msg.tic)
colNames <- c('PM2.5', 'PM10','O3')

for (s in 1:35) {
    for (col in colNames) {
        #sometimes a station may not have any data at all, skip it
        if (length(which(is.na(bj.aq.latest.filled[[s]][col]))) > nrow(bj.aq.latest.filled[[s]]) - 3) {
            bj.aq.latest.filled[[s]][col]<-lapply(bj.aq.latest.filled[[s]][col],function(x) x=0)
           next 
        }
        bj.aq.latest.filled[[s]][col] <- na.kalman(bj.aq.latest.filled[[s]][col])
    }
    bj.aq.latest.filled[[s]]$utc_time <- as.POSIXct(bj.aq.latest.filled[[s]]$utc_time, origin="1970-01-01", tz="UTC")
}

#London
colNames <- c('PM2.5', 'PM10')

for (s in 1:13) {
    for (col in colNames) {
        #sometimes a station may not have any data at all, skip it
        if (length(which(is.na(ld.aq.latest.filled[[s]][col]))) > nrow(ld.aq.latest.filled[[s]]) - 3) {
            ld.aq.latest.filled[[s]][col]<-lapply(ld.aq.latest.filled[[s]][col],function(x) x=0)
           next 
        }
        ld.aq.latest.filled[[s]][col] <- na.kalman(ld.aq.latest.filled[[s]][col])
    }
    ld.aq.latest.filled[[s]]$utc_time <- as.POSIXct(ld.aq.latest.filled[[s]]$utc_time, origin="1970-01-01", tz="UTC")
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Fill in missing data with Kalman...
INFO: Fill in missing data with Kalman: 0.28 seconds elapsed


In [12]:
# Add time features to latest data
# Function to find the season of the year given the date
find_season <- function(date) {
    spring <- paste(year(date),"-03-20", sep="")
    spring <- as.POSIXct(spring, origin="1970-01-01", tz="UTC")
    summer <- paste(year(date),"-06-21", sep="")
    summer <- as.POSIXct(summer, origin="1970-01-01", tz="UTC")
    fall <- paste(year(date),"-09-22", sep="")
    fall <- as.POSIXct(fall, origin="1970-01-01", tz="UTC")
    winter <- paste(year(date),"-12-21", sep="")
    winter <- as.POSIXct(winter, origin="1970-01-01", tz="UTC")
    if (date >= spring && date < summer )
        return(1) # Spring
    else if (date >= summer && date < fall)
        return(2) # Summer
    else if (date >= fall && date < winter)
        return(3) # Fall
    else
        return(4) # Winter    
}

In [13]:
# Functions to find last week's values for beijing and london
# from historical
bj_find_last_week_val <- function(station_index, datetime, pollutant) {
    lastweektime = datetime - 604800
    historicalLast10Day <- tail(bj.fullStationData[[station_index]],240)
    lastVal <- historicalLast10Day[[pollutant]][historicalLast10Day$utc_time==lastweektime][1]
    return (ifelse(is.na(lastVal),0,lastVal))
}

ld_find_last_week_val <- function(station_index, datetime, pollutant) {
    lastweektime = datetime - 604800
    historicalLast10Day <- tail(ld.fullStationData[[station_index]],240)
    lastVal <- historicalLast10Day[[pollutant]][historicalLast10Day$utc_time==lastweektime][1]
    return (ifelse(is.na(lastVal),0,lastVal))
}

# Functions to find two days ago's values for beijing and london
bj_find_two_day_val <- function(station_index, datetime, pollutant, aq_latest) {
    twodaytime = datetime - 172800
    historicalLast5Day <- tail(bj.fullStationData[[station_index]],120)
    lastVal <- historicalLast5Day[[pollutant]][historicalLast5Day$utc_time==twodaytime][1]
    if (is.na(lastVal)) {
        lastVal <- bj.aq.latest.filled[[station_index]][[pollutant]][bj.aq.latest.filled[[station_index]]$utc_time==twodaytime][1]
    }
    return (ifelse(is.na(lastVal),0,lastVal))
}

ld_find_two_day_val <- function(station_index, datetime, pollutant) {
    twodaytime = datetime - 172800
    historicalLast5Day <- tail(ld.fullStationData[[station_index]],120)
    lastVal <- historicalLast5Day[[pollutant]][historicalLast5Day$utc_time==twodaytime][1]
    if (is.na(lastVal)) {
        lastVal <- ld.aq.latest.filled[[station_index]][[pollutant]][ld.aq.latest.filled[[station_index]]$utc_time==twodaytime][1]
    }
    return (ifelse(is.na(lastVal),0,lastVal))
}

In [14]:
#add time features to latest beijing data
# We need the time only once, for all bj stations
tic("Add time features to latest Beijing data", quiet = FALSE, func.tic = my.msg.tic)
totalRows <- nrow(bj.aq.latest.filled[[1]])
bj.timeVector <- as.data.frame(matrix(nrow = totalRows, ncol = 5)) 
names(bj.timeVector) <- c("hour", "month", "weekday", "year", "season")
for (r in 1:totalRows) {
    t <- bj.aq.latest.filled[[1]]$utc_time[r]
    bj.timeVector$hour[r] <- hour(t)
    bj.timeVector$month[r] <- month(t)
    bj.timeVector$weekday[r] <- wday(t)
    bj.timeVector$year[r] <- year(t)
    bj.timeVector$season[r] <- find_season(t)
}


for (s in 1:35) {
    for (r in 1:nrow(bj.aq.latest.filled[[s]])) {
        t <- bj.aq.latest.filled[[s]]$utc_time[r]
        bj.aq.latest.filled[[s]]$hour[r] <- bj.timeVector$hour[r]
        bj.aq.latest.filled[[s]]$month[r] <- bj.timeVector$month[r]
        bj.aq.latest.filled[[s]]$weekday[r] <- bj.timeVector$weekday[r]
        bj.aq.latest.filled[[s]]$year[r] <- bj.timeVector$year[r]
        bj.aq.latest.filled[[s]]$season[r] <- bj.timeVector$season[r]
        bj.aq.latest.filled[[s]]$lwPM2.5[r] <- bj_find_last_week_val(s,t,"PM2.5")
        bj.aq.latest.filled[[s]]$lwPM10[r] <- bj_find_last_week_val(s,t,"PM10")
        bj.aq.latest.filled[[s]]$lwO3[r] <- bj_find_last_week_val(s,t,"O3")
        bj.aq.latest.filled[[s]]$tdPM2.5[r] <- bj_find_two_day_val(s,t,"PM2.5")
        bj.aq.latest.filled[[s]]$tdPM10[r] <- bj_find_two_day_val(s,t,"PM10")
        bj.aq.latest.filled[[s]]$tdO3[r] <- bj_find_two_day_val(s,t,"O3")   
    }
}
rm(bj.timeVector)
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Add time features to latest Beijing data...
INFO: Add time features to latest Beijing data: 45.099 seconds elapsed


In [15]:
#add time features to london data
# We need the time only once, for all ld stations
tic("Add time features to latest London data", quiet = FALSE, func.tic = my.msg.tic)
totalRows <- nrow(ld.aq.latest.filled[[1]])
ld.timeVector <- as.data.frame(matrix(nrow = totalRows, ncol = 5)) 
names(ld.timeVector) <- c("hour", "month", "weekday", "year", "season")
for (r in 1:totalRows) {
    t <- ld.aq.latest.filled[[1]]$utc_time[r]
    ld.timeVector$hour[r] <- hour(t)
    ld.timeVector$month[r] <- month(t)
    ld.timeVector$weekday[r] <- wday(t)
    ld.timeVector$year[r] <- year(t)
    ld.timeVector$season[r] <- find_season(t)
}

for (s in 1:13) {
    for (r in 1:nrow(ld.aq.latest.filled[[s]])) {
        t <- ld.aq.latest.filled[[s]]$utc_time[r]
        ld.aq.latest.filled[[s]]$hour[r] <- ld.timeVector$hour[r]
        ld.aq.latest.filled[[s]]$month[r] <- ld.timeVector$month[r]
        ld.aq.latest.filled[[s]]$weekday[r] <- ld.timeVector$weekday[r]
        ld.aq.latest.filled[[s]]$year[r] <- ld.timeVector$year[r]
        ld.aq.latest.filled[[s]]$season[r] <- ld.timeVector$season[r]
        ld.aq.latest.filled[[s]]$lwPM2.5[r] <- ld_find_last_week_val(s,t,"PM2.5")
        ld.aq.latest.filled[[s]]$lwPM10[r] <- ld_find_last_week_val(s,t,"PM10")
        ld.aq.latest.filled[[s]]$tdPM2.5[r] <- ld_find_two_day_val(s,t,"PM2.5")
        ld.aq.latest.filled[[s]]$tdPM10[r] <- ld_find_two_day_val(s,t,"PM10")
    }
}
rm(ld.timeVector)
gc()
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Add time features to latest London data...


,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,945391,50.5,2011837,107.5,NA,2011837,107.5
Vcells,13517651,103.2,25872272,197.4,16384,17662943,134.8


INFO: Add time features to latest London data: 13.363 seconds elapsed


In [16]:
# Add meo features to latest data
# Load file to df
bj.meo.latest = read.data('bj_grid_meteorology_latest.csv')
bj_aq_grid = read.data('beijing_aq_grid_relation.csv')
ld.meo.latest = read.data('ld_grid_meteorology_latest.csv')
ld_aq_grid = read.data('london_aq_grid_relation.csv')

In [17]:
# Convert time to POSIX
bj.meo.latest$time <- as.POSIXct(bj.meo.latest$time, format="%Y-%m-%d %H:%M:%S", tz="UTC")
ld.meo.latest$time <- as.POSIXct(ld.meo.latest$time, format="%Y-%m-%d %H:%M:%S", tz="UTC")

In [18]:
# Functions that get weather features
getTemp <- function(meo.latest, gridName, time) {
    temp <- meo.latest$temperature[meo.latest$station_id==gridName & meo.latest$time==time][1]
    if (is.na(temp)) {
        temp <- 20
    }
    return (temp)
}

getHumidity <- function(meo.latest, gridName, time) {
    humidity <- meo.latest$humidity[meo.latest$station_id==gridName & meo.latest$time==time][1]
    if (is.na(humidity)) {
        humidity <- 22
    }
    return (humidity)
}

getPressure <- function(meo.latest, gridName, time) {
    pressure <- meo.latest$pressure[meo.latest$station_id==gridName & meo.latest$time==time][1]
    if (is.na(pressure)) {
        pressure <- 940
    }
    return (pressure)
}

getWindSpeed <- function(meo.latest, gridName, time) {
    windSpeed <- meo.latest$wind_speed[meo.latest$station_id==gridName & meo.latest$time==time][1]
    if (is.na(windSpeed)) {
        windSpeed <- 18
    }
    return (windSpeed)
}

getWindDir <- function(meo.latest, gridName, time) {
    windDir <- meo.latest$wind_direction[meo.latest$station_id==gridName & meo.latest$time==time][1]
    if (is.na(windDir)) {
        return ("NO")
    }
    windDir <- as.numeric(windDir)
    if (windDir > 337 & windDir<361 | windDir >=0 & windDir <= 22) {
        windDir <- "N"
    } else if (windDir > 22 & windDir <= 67) {
        windDir <- "NE"
    } else if (windDir > 67 & windDir <= 112) {
        windDir <- "E"
    }  else if (windDir > 112 & windDir <= 157) {
        windDir <- "SE"
    }  else if (windDir > 157 & windDir <= 202) {
        windDir <- "S"
    }  else if (windDir > 202 & windDir <= 247) {
        windDir <- "SW"
    }  else if (windDir > 247 & windDir <= 292) {
        windDir <- "W"
    }  else if (windDir > 292 & windDir <= 337) {
        windDir <- "NW"
    }  else {
        windDir <- "NO"
    }  
    return (windDir)
}

In [19]:
tic("Add meo to latest bj data", quiet = FALSE, func.tic = my.msg.tic)
# Add the meo features to the bj latest data
for (s in 1:18) {
    # First we have to find the 4 closest grid meo stations to the aq station
    aqName <- bj_station_names_vec[s]
    ULGridName <- bj_aq_grid$ULGridName[bj_aq_grid$station_id==aqName]
    URGridName <- bj_aq_grid$URGridName[bj_aq_grid$station_id==aqName]
    LLGridName <- bj_aq_grid$LLGridName[bj_aq_grid$station_id==aqName]
    LRGridName <- bj_aq_grid$LRGridName[bj_aq_grid$station_id==aqName]
    bj.aq.latest.filled[[s]]$utc_time <- as.POSIXct(bj.aq.latest.filled[[s]]$utc_time, origin="1970-01-01", tz="UTC")
    
    tempVec <- NULL
    humidityVec <- NULL
    pressureVec <- NULL
    windSpeedVec <- NULL
    windDirVec <- NULL
    
    for (i in 1:nrow(bj.aq.latest.filled[[s]])) {
        time <- bj.aq.latest.filled[[s]]$utc_time[i]
        tempVec[i] <- mean(c(getTemp(bj.meo.latest, ULGridName, time), getTemp(bj.meo.latest, URGridName, time), getTemp(bj.meo.latest, LLGridName, time), getTemp(bj.meo.latest, LRGridName, time)))
        pressureVec[i] <- mean(c(getPressure(bj.meo.latest, ULGridName, time), getPressure(bj.meo.latest, URGridName, time), getPressure(bj.meo.latest, LLGridName, time), getPressure(bj.meo.latest, LRGridName, time)))
        humidityVec[i] <- mean(c(getHumidity(bj.meo.latest, ULGridName, time), getHumidity(bj.meo.latest, URGridName, time), getHumidity(bj.meo.latest, LLGridName, time), getHumidity(bj.meo.latest, LRGridName, time)))        
        windSpeedVec[i] <- mean(c(getWindSpeed(bj.meo.latest, ULGridName, time), getWindSpeed(bj.meo.latest, URGridName, time), getWindSpeed(bj.meo.latest, LLGridName, time), getWindSpeed(bj.meo.latest, LRGridName, time)))
        windDirVec[i] <- getWindDir(bj.meo.latest, ULGridName, time)
    }
    
    bj.aq.latest.filled[[s]]$temperature <- tempVec
    bj.aq.latest.filled[[s]]$pressure <- pressureVec
    bj.aq.latest.filled[[s]]$humidity <- humidityVec
    bj.aq.latest.filled[[s]]$windSpeed <- windSpeedVec
    bj.aq.latest.filled[[s]]$windDir <- windDirVec
        
}
for (s in 19:35) {
    # First we have to find the 4 closest grid meo stations to the aq station
    aqName <- bj_station_names_vec[s]
    ULGridName <- bj_aq_grid$ULGridName[bj_aq_grid$station_id==aqName]
    URGridName <- bj_aq_grid$URGridName[bj_aq_grid$station_id==aqName]
    LLGridName <- bj_aq_grid$LLGridName[bj_aq_grid$station_id==aqName]
    LRGridName <- bj_aq_grid$LRGridName[bj_aq_grid$station_id==aqName]
    bj.aq.latest.filled[[s]]$utc_time <- as.POSIXct(bj.aq.latest.filled[[s]]$utc_time, origin="1970-01-01", tz="UTC")
    
    tempVec <- NULL
    humidityVec <- NULL
    pressureVec <- NULL
    windSpeedVec <- NULL
    windDirVec <- NULL
    
    for (i in 1:nrow(bj.aq.latest.filled[[s]])) {
        time <- bj.aq.latest.filled[[s]]$utc_time[i]
        tempVec[i] <- mean(c(getTemp(bj.meo.latest, ULGridName, time), getTemp(bj.meo.latest, URGridName, time), getTemp(bj.meo.latest, LLGridName, time), getTemp(bj.meo.latest, LRGridName, time)))
        pressureVec[i] <- mean(c(getPressure(bj.meo.latest, ULGridName, time), getPressure(bj.meo.latest, URGridName, time), getPressure(bj.meo.latest, LLGridName, time), getPressure(bj.meo.latest, LRGridName, time)))
        humidityVec[i] <- mean(c(getHumidity(bj.meo.latest, ULGridName, time), getHumidity(bj.meo.latest, URGridName, time), getHumidity(bj.meo.latest, LLGridName, time), getHumidity(bj.meo.latest, LRGridName, time)))        
        windSpeedVec[i] <- mean(c(getWindSpeed(bj.meo.latest, ULGridName, time), getWindSpeed(bj.meo.latest, URGridName, time), getWindSpeed(bj.meo.latest, LLGridName, time), getWindSpeed(bj.meo.latest, LRGridName, time)))
        windDirVec[i] <- getWindDir(bj.meo.latest, ULGridName, time)
    }
    
    bj.aq.latest.filled[[s]]$temperature <- tempVec
    bj.aq.latest.filled[[s]]$pressure <- pressureVec
    bj.aq.latest.filled[[s]]$humidity <- humidityVec
    bj.aq.latest.filled[[s]]$windSpeed <- windSpeedVec
    bj.aq.latest.filled[[s]]$windDir <- windDirVec
        
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Add meo to latest bj data...
INFO: Add meo to latest bj data: 12.486 seconds elapsed


In [20]:
tic("Add meo to latest ld data", quiet = FALSE, func.tic = my.msg.tic)
# Add the meo features to the ld latest data
for (s in 1:13) {
    # First we have to find the 4 closest grid meo stations to the aq station
    aqName <- ld_station_names_vec[s]
    ULGridName <- ld_aq_grid$ULGridName[ld_aq_grid$station_id==aqName]
    URGridName <- ld_aq_grid$URGridName[ld_aq_grid$station_id==aqName]
    LLGridName <- ld_aq_grid$LLGridName[ld_aq_grid$station_id==aqName]
    LRGridName <- ld_aq_grid$LRGridName[ld_aq_grid$station_id==aqName]
    ld.aq.latest.filled[[s]]$utc_time <- as.POSIXct(ld.aq.latest.filled[[s]]$utc_time, origin="1970-01-01", tz="UTC")
    
    tempVec <- NULL
    humidityVec <- NULL
    pressureVec <- NULL
    windSpeedVec <- NULL
    windDirVec <- NULL
    
    for (i in 1:nrow(ld.aq.latest.filled[[s]])) {
        time <- ld.aq.latest.filled[[s]]$utc_time[i]
        tempVec[i] <- mean(c(getTemp(ld.meo.latest, ULGridName, time), getTemp(ld.meo.latest, URGridName, time), getTemp(ld.meo.latest, LLGridName, time), getTemp(ld.meo.latest, LRGridName, time)))
        pressureVec[i] <- mean(c(getPressure(ld.meo.latest, ULGridName, time), getPressure(ld.meo.latest, URGridName, time), getPressure(ld.meo.latest, LLGridName, time), getPressure(ld.meo.latest, LRGridName, time)))
        humidityVec[i] <- mean(c(getHumidity(ld.meo.latest, ULGridName, time), getHumidity(ld.meo.latest, URGridName, time), getHumidity(ld.meo.latest, LLGridName, time), getHumidity(ld.meo.latest, LRGridName, time)))
        windSpeedVec[i] <- mean(c(getWindSpeed(ld.meo.latest, ULGridName, time), getWindSpeed(ld.meo.latest, URGridName, time), getWindSpeed(ld.meo.latest, LLGridName, time), getWindSpeed(ld.meo.latest, LRGridName, time)))
        windDirVec[i] <- getWindDir(ld.meo.latest, ULGridName, time)
    }
    
    ld.aq.latest.filled[[s]]$temperature <- tempVec
    ld.aq.latest.filled[[s]]$pressure <- pressureVec
    ld.aq.latest.filled[[s]]$humidity <- humidityVec
    ld.aq.latest.filled[[s]]$windSpeed <- windSpeedVec
    ld.aq.latest.filled[[s]]$windDir <- windDirVec
        
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Add meo to latest ld data...
INFO: Add meo to latest ld data: 7.992 seconds elapsed


In [21]:
# combine BJ historical and latest
tic("Combine latest with Historical data", quiet = FALSE, func.tic = my.msg.tic)
for (s in 1:35) {
    bj.fullStationData[[s]]$utc_time <- as.POSIXct(bj.fullStationData[[s]]$utc_time, origin="1970-01-01", format="%Y-%m-%d %H:%M:%S", tz="UTC")
    bj.fullStationData[[s]] <- rbind(bj.fullStationData[[s]], bj.aq.latest.filled[[s]])
    bj.fullStationData[[s]] <- bj.fullStationData[[s]][order(bj.fullStationData[[s]]$stationId, bj.fullStationData[[s]]$utc_time),]
    bj.fullStationData[[s]] <- bj.fullStationData[[s]][!duplicated(bj.fullStationData[[s]][,c('utc_time')]),]   
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Combine latest with Historical data...
INFO: Combine latest with Historical data: 0.685 seconds elapsed


In [22]:
# combine London historical and latest
tic("Combine London latest with Historical data", quiet = FALSE, func.tic = my.msg.tic)
for (s in 1:13) {
    ld.fullStationData[[s]]$utc_time <- as.POSIXct(ld.fullStationData[[s]]$utc_time, origin="1970-01-01", format="%Y-%m-%d %H:%M:%S", tz="UTC")
    ld.fullStationData[[s]] <- rbind(ld.fullStationData[[s]], ld.aq.latest.filled[[s]])
    ld.fullStationData[[s]] <- ld.fullStationData[[s]][order(ld.fullStationData[[s]]$stationId, ld.fullStationData[[s]]$utc_time),]
    ld.fullStationData[[s]] <- ld.fullStationData[[s]][!duplicated(ld.fullStationData[[s]][,c('utc_time')]),]   
}

#rm(bj.aq.latest.filled)
#rm(ld.aq.latest.filled)
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Combine London latest with Historical data...
INFO: Combine London latest with Historical data: 0.218 seconds elapsed


In [23]:
tic("Convert categorical to factors", quiet = FALSE, func.tic = my.msg.tic)
# convert categorical variables to factors
for (s in 1:35) {
    bj.fullStationData[[s]]$year <- as.factor(bj.fullStationData[[s]]$year)
    bj.fullStationData[[s]]$season <- as.factor(bj.fullStationData[[s]]$season)
    bj.fullStationData[[s]]$month <- as.factor(bj.fullStationData[[s]]$month)
    bj.fullStationData[[s]]$weekday <- as.factor(bj.fullStationData[[s]]$weekday)
    bj.fullStationData[[s]]$hour <- as.factor(bj.fullStationData[[s]]$hour)
    bj.fullStationData[[s]]$windDir <- as.factor(bj.fullStationData[[s]]$windDir)
}

for (s in 1:13) {
    ld.fullStationData[[s]]$year <- as.factor(ld.fullStationData[[s]]$year)
    ld.fullStationData[[s]]$season <- as.factor(ld.fullStationData[[s]]$season)
    ld.fullStationData[[s]]$month <- as.factor(ld.fullStationData[[s]]$month)
    ld.fullStationData[[s]]$weekday <- as.factor(ld.fullStationData[[s]]$weekday)
    ld.fullStationData[[s]]$hour <- as.factor(ld.fullStationData[[s]]$hour)
    ld.fullStationData[[s]]$windDir <- as.factor(ld.fullStationData[[s]]$windDir)
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Convert categorical to factors...
INFO: Convert categorical to factors: 1.765 seconds elapsed


In [24]:
# Remove all negative numbers and replace with 0
tic("Remove all negative numbers and replace with 0", quiet = FALSE, func.tic = my.msg.tic)
for (s in 1:35) {
    for (i in 1:nrow(bj.fullStationData[[s]])) {
        if (bj.fullStationData[[s]]$PM2.5[i] < 0) {
            bj.fullStationData[[s]]$PM2.5[i] <- 0
        }
        if (bj.fullStationData[[s]]$PM10[i] < 0) {
            bj.fullStationData[[s]]$PM10[i] <- 0
        }
        if (bj.fullStationData[[s]]$O3[i] < 0) {
            bj.fullStationData[[s]]$O3[i] <- 0
        }
        if (bj.fullStationData[[s]]$lwPM2.5[i] < 0) {
            bj.fullStationData[[s]]$lwPM2.5[i] <- 0
        }
        if (bj.fullStationData[[s]]$lwPM10[i] < 0) {
            bj.fullStationData[[s]]$lwPM10[i] <- 0
        }
        if (bj.fullStationData[[s]]$lwO3[i] < 0) {
            bj.fullStationData[[s]]$lwO3[i] <- 0
        }
    }
}

for (s in 1:13) {
    for (i in 1:nrow(ld.fullStationData[[s]])) {
        if (ld.fullStationData[[s]]$PM2.5[i] < 0) {
            ld.fullStationData[[s]]$PM2.5[i] <- 0
        }
        if (ld.fullStationData[[s]]$PM10[i] < 0) {
            ld.fullStationData[[s]]$PM10[i] <- 0
        }
        if (ld.fullStationData[[s]]$lwPM2.5[i] < 0) {
            ld.fullStationData[[s]]$lwPM2.5[i] <- 0
        }
        if (ld.fullStationData[[s]]$lwPM10[i] < 0) {
            ld.fullStationData[[s]]$lwPM10[i] <- 0
        }
    }
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Remove all negative numbers and replace with 0...
INFO: Remove all negative numbers and replace with 0: 18.062 seconds elapsed


In [25]:
# Check the time diff and nrow to be equal for the merged dataset
bj_station_starting_times <- list()
bj_station_ending_times <- list()
bj_station_diff_times <- list()
for (s in 1:35) {
    nrow(bj.fullStationData[[s]])
    bj_station_starting_times[[s]] <- head(bj.fullStationData[[s]]$utc_time,1)
    bj_station_ending_times[[s]] <- tail(bj.fullStationData[[s]]$utc_time,1)

    # Find the difference between the latest bj first and last record
    bj_station_diff_times[[s]] <- as.integer(bj_station_ending_times[[s]] - bj_station_starting_times[[s]] + 1) * 24
    - hour(bj_station_starting_times[[s]]) - (23 - hour(bj_station_ending_times[[s]]))
}

# Repeat for London
ld_station_starting_times <- list()
ld_station_ending_times <- list()
ld_station_diff_times <- list()
for (s in 1:13) {
    nrow(ld.fullStationData[[s]])
    ld_station_starting_times[[s]] <- head(ld.fullStationData[[s]]$utc_time,1)
    ld_station_ending_times[[s]] <- tail(ld.fullStationData[[s]]$utc_time,1)
    ld_station_diff_times[[s]] <- as.integer(ld_station_ending_times[[s]] - ld_station_starting_times[[s]] + 1) * 24
    - hour(ld_station_starting_times[[s]]) - (23 - hour(ld_station_ending_times[[s]]))
}

bj_diff <- data.frame(matrix(unlist(bj_station_diff_times), nrow=35, byrow=T))
ld_diff <- data.frame(matrix(unlist(ld_station_diff_times), nrow=13, byrow=T))

if (sd(bj_diff[,1])!=0 || sd(ld_diff[,1]!=0)) {
    print ("Time mismatch detected between BJ stations")
}

bj.startingTime <- bj_station_starting_times[[1]]
print(paste("bj starting time:",bj.startingTime))
bj.endingTime <- bj_station_ending_times[[1]]
print(paste("bj ending time:",bj.endingTime))
ld.startingTime <- ld_station_starting_times[[1]]
print(paste("ld starting time:",ld.startingTime))
ld.endingTime <- ld_station_ending_times[[1]]
print(paste("ld ending time:",ld.endingTime))
# Clear lists from memory
rm(bj_station_starting_times)
rm(bj_station_ending_times)
rm(bj_station_diff_times)
rm(ld_station_starting_times)
rm(ld_station_ending_times)
rm(ld_station_diff_times)
rm(bj_diff)
rm(ld_diff)
gc()

[1] "bj starting time: 2017-01-10 14:00:00"
[1] "bj ending time: 2018-05-31 22:00:00"
[1] "ld starting time: 2017-01-10"
[1] "ld ending time: 2018-05-31 21:00:00"


,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,943335,50.4,2011837,107.5,NA,2011837,107.5
Vcells,14107038,107.7,25872272,197.4,16384,25872271,197.4


In [26]:
########################################################################
## This is only for testing
########################################################################
# Create dataframe that will hold the last 48 hour results
if (isValidation==1) {
    tic("Remove two days from training set for validation", quiet = FALSE, func.tic = my.msg.tic)
    actual48Data <- as.data.frame(matrix(nrow = 2304, ncol = 4))
    names(actual48Data) <- c("test_id", "PM2.5", "PM10", "O3")
    for (s in 1:35) {
        last48rows <- tail(bj.fullStationData[[s]],48)
        for (t in 1:48) {
           actual48Data[(s-1)*48+t,] <- c(paste(bj_station_names_vec[s], "#", t-1, sep=""),last48rows$PM2.5[t],
                                        last48rows$PM10[t],last48rows$O3[t])
        }
        bj.fullStationData[[s]] <- head(bj.fullStationData[[s]],-48)
    }

    for (s in 1:13) {
        last48rows <- tail(ld.fullStationData[[s]],48)
        for (t in 1:48) {
           actual48Data[(s-1)*48+t+1680,] <- c(paste(ld_station_names_vec[s], "#", t-1, sep=""),last48rows$PM2.5[t],
                                        last48rows$PM10[t],0)
        }
        ld.fullStationData[[s]] <- head(ld.fullStationData[[s]],-48)
    }

    write.csv(actual48Data, file = "Validation48Data.csv", row.names=FALSE)
    toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")
}

In [27]:
############################################
# Model Training Section
############################################

In [28]:
# Train the Random Forest models for each station
tic("Train RF for Beijing", quiet = FALSE, func.tic = my.msg.tic)

if (trainRF == 1) 
{
    
for (s in 1:18) {
    #save each trained model to file
    stationName <- bj.fullStationData[[s]]$stationId[1]
    modelfile <- paste("rf_models/",stationName,"_rf_PM25.RData",sep="")
    rf_modelPM2.5 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM2.5","tdPM2.5")], y = bj.fullStationData[[s]]$PM2.5, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM2.5, file = modelfile)
    rm(rf_modelPM2.5)
    
    modelfile <- paste("rf_models/",stationName,"_rf_PM10.RData",sep="")
    rf_modelPM10 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM10","tdPM10")], y = bj.fullStationData[[s]]$PM10, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM10, file = modelfile)
    rm(rf_modelPM10)
    
    modelfile <- paste("rf_models/",stationName,"_rf_O3.RData",sep="")
    rf_modelO3 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwO3","tdO3")], y = bj.fullStationData[[s]]$O3, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)  
    save(rf_modelO3, file = modelfile)
    rm(rf_modelO3)
}

for (s in 19:35) {
    #save each trained model to file
    stationName <- bj.fullStationData[[s]]$stationId[1]
    modelfile <- paste("rf_models/",stationName,"_rf_PM25.RData",sep="")
    rf_modelPM2.5 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM2.5","tdPM2.5")], y = bj.fullStationData[[s]]$PM2.5, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM2.5, file = modelfile)
    rm(rf_modelPM2.5)
    
    modelfile <- paste("rf_models/",stationName,"_rf_PM10.RData",sep="")
    rf_modelPM10 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM10","tdPM10")], y = bj.fullStationData[[s]]$PM10, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM10, file = modelfile)
    rm(rf_modelPM10)
    
    modelfile <- paste("rf_models/",stationName,"_rf_O3.RData",sep="")
    rf_modelO3 <- randomForest(bj.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwO3","tdO3")], y = bj.fullStationData[[s]]$O3, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)  
    save(rf_modelO3, file = modelfile)
    rm(rf_modelO3)
}

}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")


Starting Train RF for Beijing...
INFO: Train RF for Beijing: 0.004 seconds elapsed


In [29]:
# Train the Random Forest models for each station
tic("Train RF for London", quiet = FALSE, func.tic = my.msg.tic)

if (trainRF == 1) 
{
    
for (s in 1:13) {
    #save each trained model to file
    stationName <- ld.fullStationData[[s]]$stationId[1]
    modelfile <- paste("rf_models/",stationName,"_rf_PM25.RData",sep="")
    rf_modelPM2.5 <- randomForest(ld.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM2.5","tdPM2.5")], y = ld.fullStationData[[s]]$PM2.5, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM2.5, file = modelfile)
    rm(rf_modelPM2.5)
    
    modelfile <- paste("rf_models/",stationName,"_rf_PM10.RData",sep="")
    rf_modelPM10 <- randomForest(ld.fullStationData[[s]][,c("hour","month","weekday","year","season","temperature","pressure","humidity","windSpeed","windDir","lwPM10","tdPM10")], y = ld.fullStationData[[s]]$PM10, ntree = ntree, mtry = mtry, 
                         nodesize=5, importance=TRUE)
    save(rf_modelPM10, file = modelfile)
    rm(rf_modelPM10)
}
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")


Starting Train RF for London...
INFO: Train RF for London: 0.005 seconds elapsed


In [30]:
##################################
# SVR Model
#################################
library(e1071)
tic("Train SVR for Beijing", quiet = FALSE, func.tic = my.msg.tic)
if (trainSVR == 1) {
    
# Train the SVR models for each station

for (s in 1:35) {
    #save each trained model to file
    stationName <- bj.fullStationData[[s]]$stationId[1]
    modelfile <- paste("svr_models/",stationName,"_svr_PM25.RData",sep="")
    svr_modelPM2.5 <- svm(PM2.5 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM2.5+tdPM2.5, data = bj.fullStationData[[s]], epsilon = 0.08, cost = 4, kernel = "radial", gamma = 0.2, type="eps-regression")
    save(svr_modelPM2.5, file = modelfile)
    rm(svr_modelPM2.5)
    
    modelfile <- paste("svr_models/",stationName,"_svr_PM10.RData",sep="")
    svr_modelPM10 <- svm(PM10 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM10+tdPM10, data = bj.fullStationData[[s]], epsilon = 0.08, cost = 4, kernel = "radial", gamma = 0.2, type="eps-regression")
    save(svr_modelPM10, file = modelfile)
    rm(svr_modelPM10)
    
    modelfile <- paste("svr_models/",stationName,"_svr_O3.RData",sep="")
    svr_modelO3 <- svm(O3 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwO3+tdO3, data = bj.fullStationData[[s]], epsilon = 0.08, cost = 4, kernel = "radial", gamma = 0.2, type="eps-regression")
    save(svr_modelO3, file = modelfile)
    rm(svr_modelO3)
}
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")    


Starting Train SVR for Beijing...
INFO: Train SVR for Beijing: 0.003 seconds elapsed


In [31]:
tic("Train SVR for London", quiet = FALSE, func.tic = my.msg.tic)
if (trainSVR == 1) {
    
# Train the SVR models for each station
for (s in 1:13) {
    #save each trained model to file
    stationName <- ld.fullStationData[[s]]$stationId[1]
    modelfile <- paste("svr_models/",stationName,"_svr_PM25.RData",sep="")
    svr_modelPM2.5 <- svm(PM2.5 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM2.5+tdPM2.5, data = ld.fullStationData[[s]], epsilon = 0.08, cost = 4, kernel = "radial", gamma = 0.2, type="eps-regression")
    save(svr_modelPM2.5, file = modelfile)
    rm(svr_modelPM2.5)
    
    modelfile <- paste("svr_models/",stationName,"_svr_PM10.RData",sep="")
    svr_modelPM10 <- svm(PM10 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM10+tdPM10, data = ld.fullStationData[[s]], epsilon = 0.08, cost = 4, kernel = "radial", gamma = 0.2, type="eps-regression")
    save(svr_modelPM10, file = modelfile)
    rm(svr_modelPM10)
}
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")  

Starting Train SVR for London...
INFO: Train SVR for London: 0.003 seconds elapsed


In [32]:
tic("scale data for Beijing ANN", quiet = FALSE, func.tic = my.msg.tic)
bj_scale_col <- c("temperature", "pressure", "humidity", "windSpeed", "lwPM2.5", "lwPM10", "lwO3", "tdPM2.5", "tdPM10", "tdO3")
if (trainANN == 1) {
    bj.scaled <- list()
    
#     for (s in 1:35) {
#         bj.fullStationData[[s]]$hour <- as.numeric(as.character(bj.fullStationData[[s]]$hour))
#         bj.fullStationData[[s]]$month <- as.numeric(as.character(bj.fullStationData[[s]]$month))
#         bj.fullStationData[[s]]$weekday <- as.numeric(as.character(bj.fullStationData[[s]]$weekday))
#         bj.fullStationData[[s]]$year <- as.numeric(as.character(bj.fullStationData[[s]]$year))
#         bj.fullStationData[[s]]$season <- as.numeric(as.character(bj.fullStationData[[s]]$season))
#         bj.fullStationData[[s]]$windDir <- as.numeric(bj.fullStationData[[s]]$windDir)
#     }
    
    for (s in 1:35) {
        bj.scaled[[s]] <- bj.fullStationData[[s]]
        bj.scaled[[s]][,bj_scale_col] <- scale(bj.scaled[[s]][,bj_scale_col])
    }
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")    

Starting scale data for Beijing ANN...
INFO: scale data for Beijing ANN: 0.004 seconds elapsed


In [33]:
tic("scale data for London ANN", quiet = FALSE, func.tic = my.msg.tic)
ld_scale_col <- c("temperature", "pressure", "humidity", "windSpeed", "lwPM2.5", "lwPM10", "tdPM2.5", "tdPM10")
if (trainANN == 1) {
    ld.scaled <- list()
    
#     for (s in 1:13) {
#         ld.fullStationData[[s]]$hour <- as.numeric(as.character(ld.fullStationData[[s]]$hour))
#         ld.fullStationData[[s]]$month <- as.numeric(as.character(ld.fullStationData[[s]]$month))
#         ld.fullStationData[[s]]$weekday <- as.numeric(as.character(ld.fullStationData[[s]]$weekday))
#         ld.fullStationData[[s]]$year <- as.numeric(as.character(ld.fullStationData[[s]]$year))
#         ld.fullStationData[[s]]$season <- as.numeric(as.character(ld.fullStationData[[s]]$season))
#         ld.fullStationData[[s]]$windDir <- as.numeric(ld.fullStationData[[s]]$windDir)
#     }
    
    for (s in 1:13) {
        ld.scaled[[s]] <- ld.fullStationData[[s]]
        ld.scaled[[s]][,ld_scale_col] <- scale(ld.scaled[[s]][,ld_scale_col])
    }
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO") 

Starting scale data for London ANN...
INFO: scale data for London ANN: 0.005 seconds elapsed


In [34]:
##################################
# ANN Model
#################################
library(neuralnet)
tic("Train ANN for Beijing", quiet = FALSE, func.tic = my.msg.tic)
if (trainANN == 1) {
    
# Train the ANN models for each station

for (s in 1:1) {
    #save each trained model to file
    stationName <- bj.fullStationData[[s]]$stationId[1]
    modelfile <- paste("ann_models/",stationName,"_ann_PM25.RData",sep="")
    ann_modelPM2.5 <- neuralnet(PM2.5 ~ temperature+pressure+humidity+windSpeed+lwPM2.5+tdPM2.5, data = bj.scaled[[s]], hidden = hidden_nodes, rep = iterations, stepmax = stepmax, err.fct = error_func, learningrate = lr, linear.output = TRUE)
    save(ann_modelPM2.5, file = modelfile)
    rm(ann_modelPM2.5)
    
    modelfile <- paste("ann_models/",stationName,"_ann_PM10.RData",sep="")
    ann_modelPM10 <- neuralnet(PM10 ~ temperature+pressure+humidity+windSpeed+lwPM10+tdPM10, data = bj.scaled[[s]], hidden = hidden_nodes, rep = iterations, stepmax = stepmax, err.fct = error_func, learningrate = lr, linear.output = TRUE)
    save(ann_modelPM10, file = modelfile)
    rm(ann_modelPM10)
    
    modelfile <- paste("ann_models/",stationName,"_ann_O3.RData",sep="")
    ann_modelO3 <- neuralnet(O3 ~ temperature+pressure+humidity+windSpeed+lwO3+tdO3, data = bj.scaled[[s]], hidden = hidden_nodes, rep = iterations, stepmax = stepmax, err.fct = error_func, learningrate = lr, linear.output = TRUE)
    save(ann_modelO3, file = modelfile)
    rm(ann_modelO3)
}
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")    

Starting Train ANN for Beijing...
INFO: Train ANN for Beijing: 0.003 seconds elapsed


In [35]:
tic("Train ANN for London", quiet = FALSE, func.tic = my.msg.tic)
if (trainANN == 1) {
    
# Train the ANN models for each station

for (s in 1:13) {
    #save each trained model to file
    stationName <- ld.fullStationData[[s]]$stationId[1]
    modelfile <- paste("ann_models/",stationName,"_ann_PM25.RData",sep="")
    ann_modelPM2.5 <- neuralnet(PM2.5 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM2.5+tdPM2.5, data = ld.scaled[[s]], hidden = hidden_nodes, rep = iterations, stepmax = stepmax, err.fct = error_func, learningrate = lr, linear.output = TRUE)
    save(ann_modelPM2.5, file = modelfile)
    rm(ann_modelPM2.5)
    
    modelfile <- paste("ann_models/",stationName,"_ann_PM10.RData",sep="")
    ann_modelPM10 <- neuralnet(PM10 ~ hour+month+weekday+year+season+temperature+pressure+humidity+windSpeed+windDir+lwPM10+tdPM10, data = ld.scaled[[s]], hidden = hidden_nodes, rep = iterations, stepmax = stepmax, err.fct = error_func, learningrate = lr, linear.output = TRUE)
    save(ann_modelPM10, file = modelfile)
    rm(ann_modelPM10)
}
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")   

Starting Train ANN for London...
INFO: Train ANN for London: 0.003 seconds elapsed


In [36]:
library(jsonlite)
# Create dataframe that will hold the submission results
submitData <- as.data.frame(matrix(nrow = 2304, ncol = 4))
names(submitData) <- c("test_id", "PM2.5", "PM10", "O3")

In [37]:
getWindDirLabel <- function(wind_dir) {
    if (is.na(wind_dir)) {
        return ("NO")
    }
    windDir <- as.numeric(wind_dir)
    if (windDir > 337 & windDir<361 | windDir >=0 & windDir <= 22) {
        windDir <- "N"
    } else if (windDir > 22 & windDir <= 67) {
        windDir <- "NE"
    } else if (windDir > 67 & windDir <= 112) {
        windDir <- "E"
    }  else if (windDir > 112 & windDir <= 157) {
        windDir <- "SE"
    }  else if (windDir > 157 & windDir <= 202) {
        windDir <- "S"
    }  else if (windDir > 202 & windDir <= 247) {
        windDir <- "SW"
    }  else if (windDir > 247 & windDir <= 292) {
        windDir <- "W"
    }  else if (windDir > 292 & windDir <= 337) {
        windDir <- "NW"
    }  else {
        windDir <- "NO"
    }  
    return (windDir)
}

In [38]:
##################################################
# Prediction Section
##################################################
# Train the Random Forest models for each station
tic("Predict RF or SVR for Beijing", quiet = FALSE, func.tic = my.msg.tic)
#data frame of 48 rows and number of features plus 3 prediction values of columns
prediction_data <- as.data.frame(matrix(nrow = 48, ncol = 19)) 
prediction_scaled <- NULL
names(prediction_data) <- c("PM2.5", "PM10", "O3", "hour", "month", "weekday", "year", "season", "lwPM2.5", "lwPM10", "lwO3", "temperature", "pressure", "humidity", "windSpeed","windDir","tdPM2.5","tdPM10","tdO3")
# Find the starting time of the next day (00:00 UTC)
time <- tail(bj.fullStationData[[1]],1)$utc_time
time <- time + 86400
time <- time - hour(time)*3600 - minute(time)*60

# Create the time and date values
for (i in 1:48) {
    hourly_time <- time + (i-1)*3600
    prediction_data$hour[i] <- hour(hourly_time)
    prediction_data$month[i] <- month(hourly_time)
    prediction_data$weekday[i] <- wday(hourly_time)
    prediction_data$year[i] <- year(hourly_time)
    prediction_data$season[i] <- find_season(hourly_time)
}

# RF doesn't like NAs
prediction_data$PM2.5 <- 0
prediction_data$PM10 <- 0
prediction_data$O3 <- 0
#set categorical variables to match training data factors levels
if (toPredict != "ANN") {
    prediction_data$year <- factor(prediction_data$year, levels=levels(bj.fullStationData[[1]]$year))
    prediction_data$season <- factor(prediction_data$season, levels=levels(bj.fullStationData[[1]]$season))
    prediction_data$month <- factor(prediction_data$month, levels=levels(bj.fullStationData[[1]]$month))
    prediction_data$hour <- factor(prediction_data$hour, levels=levels(bj.fullStationData[[1]]$hour))
    prediction_data$weekday <- factor(prediction_data$weekday, levels=levels(bj.fullStationData[[1]]$weekday))    
} else {
    #prediction_data$year <- as.numeric(as.character(prediction_data$year))
    #prediction_data$season <- as.numeric(as.character(prediction_data$season))
    #prediction_data$month <- as.numeric(as.character(prediction_data$month))
    #prediction_data$hour <- as.numeric(as.character(prediction_data$hour))
    #prediction_data$weekday <- as.numeric(as.character(prediction_data$weekday))
    
    prediction_scaled <- prediction_data
    prediction_scaled[, c("hour", "weekday")] <- scale(prediction_scaled[,c("hour", "weekday")])
}

# For every station:
for (s in 1:35) {
    # First get the weather for the 48 hour period
    lat <- bj_aq_grid$latitude[s]
    lon <- bj_aq_grid$longitude[s]
    URL<-paste("http://api.openweathermap.org/data/2.5/forecast?lat=", lat, "&lon=", lon, "&appid=d6f07a7bf09c7b5f2d743c38aa1999e2", sep="")
    weather_data<-fromJSON(URL)
    temp_list <- weather_data[[4]]$main[,1]
    pressure_list <- weather_data[[4]]$main[,4]
    humidity_list <- weather_data[[4]]$main[,7]
    windSpeed_list <- weather_data[[4]]$wind[,1]
    windDir_list <- weather_data[[4]]$wind[,2]
    hour_list <- weather_data[[4]]$dt_txt
    hour_list <- as.POSIXct(hour_list, origin="1970-01-01", format="%Y-%m-%d %H:%M:%S", tz="UTC")
    index <- which(hour_list==time)
    temperature17 <- temp_list[index:(index+16)]
    pressure17 <- pressure_list[index:(index+16)]
    humidity17 <- humidity_list[index:(index+16)]
    windSpeed17 <- windSpeed_list[index:(index+16)]
    windDir17 <- windDir_list[index:(index+16)]
    
    for (i in 1:48) {
        hourly_time <- time + (i-1)*3600
        prediction_data$lwPM2.5[i] <- bj_find_last_week_val(s,hourly_time,"PM2.5")
        prediction_data$lwPM10[i] <- bj_find_last_week_val(s,hourly_time,"PM10")
        prediction_data$lwO3[i] <- bj_find_last_week_val(s,hourly_time,"O3")
        prediction_data$tdPM2.5[i] <- bj_find_two_day_val(s,hourly_time,"PM2.5")
        prediction_data$tdPM10[i] <- bj_find_two_day_val(s,hourly_time,"PM10")
        prediction_data$tdO3[i] <- bj_find_two_day_val(s,hourly_time,"O3")
        prediction_data$temperature[i] <- ifelse(i%%3==1, temperature17[(i+2)/3]-273.15, NA)
        prediction_data$pressure[i] <- ifelse(i%%3==1, pressure17[(i+2)/3], NA)
        prediction_data$humidity[i] <- ifelse(i%%3==1, humidity17[(i+2)/3], NA)
        prediction_data$windSpeed[i] <- ifelse(i%%3==1, windSpeed17[(i+2)/3], NA)
        if (i %% 3 == 1) {
            windDirLabel <- getWindDirLabel(windDir17[(i+2)/3])
        } else {
            windDirLabel <- prediction_data$windDir[i-1]
        }
        prediction_data$windDir[i] <- windDirLabel
    }
    
    #convert windir category to factor
    prediction_data$windDir <- factor(prediction_data$windDir, levels=levels(bj.fullStationData[[s]]$windDir))
    
    prediction_data$temperature <- na.kalman(prediction_data$temperature)
    prediction_data$pressure <- na.kalman(prediction_data$pressure)
    prediction_data$humidity <- na.kalman(prediction_data$humidity)
    prediction_data$windSpeed <- na.kalman(prediction_data$windSpeed)
    
    if (toPredict == "ANN"){
        prediction_data$windDir <- as.numeric(prediction_data$windDir)
        
        prediction_scaled[,c("windDir","lwPM2.5", "lwPM10", "lwO3", "tdPM2.5", "tdPM10", "tdO3", "temperature","pressure","humidity","windSpeed")] <-
             scale(prediction_data[,c("windDir","lwPM2.5", "lwPM10", "lwO3", "tdPM2.5", "tdPM10", "tdO3", "temperature","pressure","humidity","windSpeed")])
    }
    
    # Load the saved RF or SVR model
    stationName <- bj.fullStationData[[s]]$stationId[1]
    if (toPredict == "RF") {       
        model_name <- paste("rf_models/",stationName,"_rf_PM25.RData",sep="")
        rf_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- predict(rf_modelPM2.5, newdata=prediction_data, type='response')
    
        model_name <- paste("rf_models/",stationName,"_rf_PM10.RData",sep="")
        rf_modelPM10 <- get(load(model_name))
        pred_testPM10 <- predict(rf_modelPM10, newdata=prediction_data, type='response')
    
        model_name <- paste("rf_models/",stationName,"_rf_O3.RData",sep="")
        rf_modelO3 <- get(load(model_name))
        pred_testO3 <- predict(rf_modelO3, newdata=prediction_data, type='response')
    } 
    else if (toPredict == "SVR") {
        model_name <- paste("svr_models/",stationName,"_svr_PM25.RData",sep="")
        svr_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- predict(svr_modelPM2.5, prediction_data)
    
        model_name <- paste("svr_models/",stationName,"_svr_PM10.RData",sep="")
        svr_modelPM10 <- get(load(model_name))
        pred_testPM10 <- predict(svr_modelPM10, prediction_data)
    
        model_name <- paste("svr_models/",stationName,"_svr_O3.RData",sep="")
        svr_modelO3 <- get(load(model_name))
        pred_testO3 <- predict(svr_modelO3, prediction_data)     
    } else {
        model_name <- paste("ann_models/",stationName,"_ann_PM25.RData",sep="")
        ann_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- compute(ann_modelPM2.5, prediction_scaled[,!names(prediction_scaled) %in% c("PM2.5", "PM10", "O3", "lwPM10", "lwO3", "tdPM10", "tdO3")])$net.result
    
        model_name <- paste("ann_models/",stationName,"_ann_PM10.RData",sep="")
        ann_modelPM10 <- get(load(model_name))
        pred_testPM10 <- compute(ann_modelPM10, prediction_scaled[,!names(prediction_scaled) %in% c("PM10", "PM2.5", "O3", "lwPM2.5", "lwO3", "tdPM2.5", "tdO3")])$net.result
    
        model_name <- paste("ann_models/",stationName,"_ann_O3.RData",sep="")
        ann_modelO3 <- get(load(model_name))
        pred_testO3 <- compute(ann_modelO3, prediction_scaled[,!names(prediction_scaled) %in% c("O3", "PM10", "PM2.5", "lwPM2.5", "lwPM10", "tdPM2.5", "tdPM10")])$net.result      
    }

    for (t in 1:48) {
        PM2.5 <- pred_testPM2.5[t]
        if (PM2.5<0) {
            PM2.5 <- 0
        }
        PM10 <- pred_testPM10[t]
        if (PM10<0) {
            PM10 <- 0
        }
        O3 <- pred_testO3[t]
        if (O3<0) {
            O3 <- 0
        }
        submitData[(s-1)*48+t,] <- c(paste(bj_station_names_vec[s], "#", t-1, sep=""),PM2.5,PM10,O3)
    }

}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Predict RF or SVR for Beijing...
INFO: Predict RF or SVR for Beijing: 28.779 seconds elapsed


In [39]:
# Predict RF model for London stations
tic("Predict RF or SVR for London", quiet = FALSE, func.tic = my.msg.tic)
for (s in 1:13) {
    
    # First get the weather for the 48 hour period
    lat <- ld_aq_grid$latitude[s]
    lon <- ld_aq_grid$longitude[s]
    URL<-paste("http://api.openweathermap.org/data/2.5/forecast?lat=", lat, "&lon=", lon, "&appid=d6f07a7bf09c7b5f2d743c38aa1999e2", sep="")
    weather_data<-fromJSON(URL)
    temp_list <- weather_data[[4]]$main[,1]
    pressure_list <- weather_data[[4]]$main[,4]
    humidity_list <- weather_data[[4]]$main[,7]
    windSpeed_list <- weather_data[[4]]$wind[,1]
    windDir_list <- weather_data[[4]]$wind[,2]
    hour_list <- weather_data[[4]]$dt_txt
    hour_list <- as.POSIXct(hour_list, origin="1970-01-01", format="%Y-%m-%d %H:%M:%S", tz="UTC")
    index <- which(hour_list==time)
    temperature17 <- temp_list[index:(index+16)]
    pressure17 <- pressure_list[index:(index+16)]
    humidity17 <- humidity_list[index:(index+16)]
    windSpeed17 <- windSpeed_list[index:(index+16)]
    windDir17 <- windDir_list[index:(index+16)]
    
    for (i in 1:48) {
        hourly_time <- time + (i-1)*3600
        prediction_data$lwPM2.5[i] <- ld_find_last_week_val(s,t,"PM2.5")
        prediction_data$lwPM10[i] <- ld_find_last_week_val(s,t,"PM10")
        prediction_data$tdPM2.5[i] <- ld_find_two_day_val(s,hourly_time,"PM2.5")
        prediction_data$tdPM10[i] <- ld_find_two_day_val(s,hourly_time,"PM10")
        prediction_data$temperature[i] <- ifelse(i%%3==1, temperature17[(i+2)/3]-273.15, NA)
        prediction_data$pressure[i] <- ifelse(i%%3==1, pressure17[(i+2)/3], NA)
        prediction_data$humidity[i] <- ifelse(i%%3==1, humidity17[(i+2)/3], NA)
        prediction_data$windSpeed[i] <- ifelse(i%%3==1, windSpeed17[(i+2)/3], NA)
        if (i %% 3 == 1) {
            windDirLabel <- getWindDirLabel(windDir17[(i+2)/3])
        } else {
            windDirLabel <- prediction_data$windDir[i-1]
        }
        prediction_data$windDir[i] <- windDirLabel
    }
    
    #convert windir category to factor
    prediction_data$windDir <- factor(prediction_data$windDir, levels=levels(ld.fullStationData[[s]]$windDir))
    
    prediction_data$temperature <- na.kalman(prediction_data$temperature)
    prediction_data$pressure <- na.kalman(prediction_data$pressure)
    prediction_data$humidity <- na.kalman(prediction_data$humidity)
    prediction_data$windSpeed <- na.kalman(prediction_data$windSpeed)
    
    if (toPredict == "ANN"){
        prediction_data$windDir <- as.numeric(prediction_data$windDir)
        
        prediction_scaled[,c("windDir","lwPM2.5", "lwPM10", "tdPM2.5", "tdPM10", "temperature","pressure","humidity","windSpeed")] <-
             scale(prediction_data[,c("windDir","lwPM2.5", "lwPM10", "tdPM2.5", "tdPM10", "temperature","pressure","humidity","windSpeed")])
    }
    
    # Load the saved RF or SVR model
    stationName <- ld.fullStationData[[s]]$stationId[1]
    if (toPredict == "RF") {       
        model_name <- paste("rf_models/",stationName,"_rf_PM25.RData",sep="")
        rf_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- predict(rf_modelPM2.5, newdata=prediction_data, type='response')
    
        model_name <- paste("rf_models/",stationName,"_rf_PM10.RData",sep="")
        rf_modelPM10 <- get(load(model_name))
        pred_testPM10 <- predict(rf_modelPM10, newdata=prediction_data, type='response')
    } 
    else if (toPredict == "SVR") {
        model_name <- paste("svr_models/",stationName,"_svr_PM25.RData",sep="")
        svr_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- predict(svr_modelPM2.5, prediction_data)
    
        model_name <- paste("svr_models/",stationName,"_svr_PM10.RData",sep="")
        svr_modelPM10 <- get(load(model_name))
        pred_testPM10 <- predict(svr_modelPM10, prediction_data)
   } else {
        model_name <- paste("ann_models/",stationName,"_ann_PM25.RData",sep="")
        ann_modelPM2.5 <- get(load(model_name))
        pred_testPM2.5 <- compute(ann_modelPM2.5, prediction_scaled)$net.result
    
        model_name <- paste("ann_models/",stationName,"_ann_PM10.RData",sep="")
        ann_modelPM10 <- get(load(model_name))
        pred_testPM10 <- compute(ann_modelPM10, prediction_scaled)$net.result    
    } 
        
    for (t in 1:48) {
        PM2.5 <- pred_testPM2.5[t]
        if (PM2.5<0) {
            PM2.5 <- 0
        }
        PM10 <- pred_testPM10[t]
        if (PM10<0) {
            PM10 <- 0
        }
        submitData[(s-1)*48+t+1680,] <- c(paste(ld_station_names_vec[s], "#", t-1, sep=""),PM2.5,PM10,0)
    }
    
}
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Predict RF or SVR for London...
INFO: Predict RF or SVR for London: 9.652 seconds elapsed


In [40]:
filename <- paste("UWDS420LiangTiaoMalax_submission_",toPredict,".csv",sep="")
write.csv(submitData, file = filename, row.names=FALSE)

In [41]:
# ################################################
# Starting ETS Section
#################################################
# Find how many hours till midnight and add them to the ETS prediction length
bj.hoursUntilMidnight <- 23 - hour(bj.endingTime)
bj.hoursUntilMidnight
ld.hoursUntilMidnight <- 23 - hour(ld.endingTime)
ld.hoursUntilMidnight

[1] 1

[1] 2

In [42]:
#################################################################
# For Validation only
#################################################################
if (isValidation==1) {
    bj.endingTime <- bj.endingTime - 48*3600
    ld.endingTime <- ld.endingTime - 48*3600
}

In [43]:
# Create time series model for ETS or ARIMA prediction
tic("Create ETS time series for BJ", quiet = FALSE, func.tic = my.msg.tic)
bj_time_index <- seq(from = bj.startingTime, to = bj.endingTime, by = "hour")

for (s in 1:35) {
    bj_time_index <- seq(from = bj.startingTime, to = bj.endingTime, by = "hour") #2018-03-17 18:00
    set.seed(1)
    valuePM2.5 <- bj.fullStationData[[s]]$PM2.5
    valuePM10 <- bj.fullStationData[[s]]$PM10
    valueO3 <- bj.fullStationData[[s]]$O3
    
    eventdataPM2.5 <- xts(valuePM2.5, order.by = bj_time_index)
    eventdataPM10 <- xts(valuePM10, order.by = bj_time_index)
    eventdataO3 <- xts(valueO3, order.by = bj_time_index)
    
    ets_dataPM2.5 <- ets(eventdataPM2.5, model="AAN")
    ets_dataPM10 <- ets(eventdataPM10, model="AAN")
    ets_dataO3 <- ets(eventdataO3, model="AAN")
      
    bj.PM2.5.forecast = forecast(ets_dataPM2.5, h=bj.hoursUntilMidnight+48, method="ets")
    bj.PM10.forecast = forecast(ets_dataPM10, h=bj.hoursUntilMidnight+48, method="ets")
    bj.O3.forecast = forecast(ets_dataO3, h=bj.hoursUntilMidnight+48, method="ets")

    for (t in 1:48) {
       submitData[(s-1)*48+t,] <- c(paste(bj_station_names_vec[s], "#", t-1, sep=""),abs(bj.PM2.5.forecast$mean[t+bj.hoursUntilMidnight]),
                                    abs(bj.PM10.forecast$mean[t+bj.hoursUntilMidnight]),abs(bj.O3.forecast$mean[t+bj.hoursUntilMidnight]))
    }
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Create ETS time series for BJ...
INFO: Create ETS time series for BJ: 46.059 seconds elapsed


In [44]:
tic("Create ETS time series for LD", quiet = FALSE, func.tic = my.msg.tic)
ld_time_index <- seq(from = ld.startingTime, to = ld.endingTime, by = "hour")

for (s in 1:13) {
    ld_time_index <- seq(from = ld.startingTime, to = ld.endingTime, by = "hour") #2018-03-17 18:00
    set.seed(1)
    valuePM2.5 <- ld.fullStationData[[s]]$PM2.5
    valuePM10 <- ld.fullStationData[[s]]$PM10
    
    eventdataPM2.5 <- xts(valuePM2.5, order.by = ld_time_index)
    eventdataPM10 <- xts(valuePM10, order.by = ld_time_index)
    
    ets_dataPM2.5 <- ets(eventdataPM2.5, model="AAN")
    ets_dataPM10 <- ets(eventdataPM10, model="AAN")
       
    ld.PM2.5.forecast = forecast(ets_dataPM2.5, h=ld.hoursUntilMidnight+48, method="ets")
    ld.PM10.forecast = forecast(ets_dataPM10, h=ld.hoursUntilMidnight+48, method="ets")

    for (t in 1:48) {
       submitData[(s-1)*48+t+1680,] <- c(paste(ld_station_names_vec[s], "#", t-1, sep=""),abs(ld.PM2.5.forecast$mean[t+ld.hoursUntilMidnight]),
                                    abs(ld.PM10.forecast$mean[t+ld.hoursUntilMidnight]),0)
    }
    
}

toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")

Starting Create ETS time series for LD...
INFO: Create ETS time series for LD: 10.109 seconds elapsed


In [45]:
write.csv(submitData, file = "UWDS420LiangTiaoMalax_submission_ETS.csv", row.names=FALSE)

In [46]:
#########################################
# Update Historical Datasets - Write to Files
########################################
if (updateHistorical == 1) {

    tic("Update historical set", quiet = FALSE, func.tic = my.msg.tic)
    # Write Beijing new Historical
    for (s in 1:35) {
        outfile <- paste(writeHistoricalFolderName,"/",bj.fullStationData[[s]]$stationId[1],"_historical.csv",sep="")
        write.csv(bj.fullStationData[[s]], file = outfile, row.names=FALSE)
    }
    
    # Write London new Historical
    for (s in 1:13) {
        outfile <- paste(writeHistoricalFolderName,"/",ld.fullStationData[[s]]$stationId[1],"_historical.csv",sep="")
        write.csv(ld.fullStationData[[s]], file = outfile, row.names=FALSE)
    }
    
    toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")
}

Starting Update historical set...
INFO: Update historical set: 11.842 seconds elapsed


In [47]:
toc(log = TRUE, quiet = FALSE, func.toc = my.msg.toc, info = "INFO")
log.txt <- tic.log(format = TRUE)
log.txt

INFO: Total time: 215.858 seconds elapsed


[[1]]
[1] "Load latest data: 0.049 sec elapsed"

[[2]]
[1] "Load Historical data: 6.712 sec elapsed"

[[3]]
[1] "Fill missing rows for latest current data: 0.622 sec elapsed"

[[4]]
[1] "Fill in missing data with Kalman: 0.28 sec elapsed"

[[5]]
[1] "Add time features to latest Beijing data: 45.099 sec elapsed"

[[6]]
[1] "Add time features to latest London data: 13.363 sec elapsed"

[[7]]
[1] "Add meo to latest bj data: 12.486 sec elapsed"

[[8]]
[1] "Add meo to latest ld data: 7.992 sec elapsed"

[[9]]
[1] "Combine latest with Historical data: 0.685 sec elapsed"

[[10]]
[1] "Combine London latest with Historical data: 0.218 sec elapsed"

[[11]]
[1] "Convert categorical to factors: 1.765 sec elapsed"

[[12]]
[1] "Remove all negative numbers and replace with 0: 18.062 sec elapsed"

[[13]]
[1] "Train RF for Beijing: 0.004 sec elapsed"

[[14]]
[1] "Train RF for London: 0.005 sec elapsed"

[[15]]
[1] "Train SVR for Beijing: 0.003 sec elapsed"

[[16]]
[1] "Train SVR for London: 0.003 sec elapsed"

[[17]]
[1] "scale data for Beijing ANN: 0.004 sec elapsed"

[[18]]
[1] "scale data for London ANN: 0.005 sec elapsed"

[[19]]
[1] "Train ANN for Beijing: 0.003 sec elapsed"

[[20]]
[1] "Train ANN for London: 0.003 sec elapsed"

[[21]]
[1] "Predict RF or SVR for Beijing: 28.779 sec elapsed"

[[22]]
[1] "Predict RF or SVR for London: 9.652 sec elapsed"

[[23]]
[1] "Create ETS time series for BJ: 46.059 sec elapsed"

[[24]]
[1] "Create ETS time series for LD: 10.109 sec elapsed"

[[25]]
[1] "Update historical set: 11.842 sec elapsed"

[[26]]
[1] "Total time: 215.858 sec elapsed"